In [1]:
import numpy as np
import pandas as pd
import os
import re
import copy
import pickle
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, KFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
import plotly.express as px

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
n_splits = 5
seed_everything(SEED)

# Define function

## Feature engineer for tabular data

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

def feature_engineering_v2_tabular(df, selector=None, imputer=None, fit=True):
    df = df.loc[:, ~df.columns.duplicated()]
    if fit: 
        y = df['sii']

    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    season_cols = [col for col in df.columns if 'Season' in col]
    pciat_cols = [col for col in df.columns if 'PCIAT' in col and 'Season' not in col]
    remaining_numeric_cols = [col for col in numeric_cols if col not in pciat_cols and col not in ['sii']]
    X = df[remaining_numeric_cols]
    if np.any(np.isinf(X)):
        X = X.replace([np.inf, -np.inf], np.nan)
    if fit: 
        imputer = SimpleImputer()
        imputed_data = imputer.fit_transform(X)
        train_imputed = pd.DataFrame(imputed_data, columns=remaining_numeric_cols)
        X = train_imputed
    else:
        X = imputer.transform(X)

    if fit:
        selector = SelectKBest(score_func=f_regression, k=30)
        X_new = selector.fit_transform(X, y)
        selected_features = X.columns[selector.get_support()]
    else: 
        X_new = selector.transform(X)
        selected_features = [col for col, selected in zip(remaining_numeric_cols, selector.get_support()) if selected]
    df_selected = pd.DataFrame(X_new, columns=selected_features)
    return df_selected, selector, imputer

def feature_engineering_tabular(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['BMI_PHR'] = df['Physical-BMI'] * df['Physical-HeartRate']
    
    return df

## Feature engineer for time series data

### Method 1: Autoencoder 

In [5]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, input_dim),
            nn.Sigmoid()
        )

        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')    
    return autoencoder, scaler

def encode_data(autoencoder, scaler, df):
    df_scaled = scaler.transform(df)
    data_tensor = torch.FloatTensor(df_scaled)
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()

    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    return df_encoded

### Method 2: External knowledge

First, prepare the features used in my sleep detection model. Please refer to the implementation by [@tatamikenn](https://www.kaggle.com/tatamikenn) [here](https://www.kaggle.com/code/tatamikenn/sleep-hdcza-a-pure-heuristic-approach-lb-0-447).

This pipeline processes accelerometer data for sleep detection, utilizing time-series datasets. It generates features to identify sleep episodes, static periods, and motion patterns, inspired by @tatamikenn's implementation.

-  `transform`: this function processes input data to generate features for analysis. It breaks down the timestamp into components like year, month, day, hour, and weekday. It also groups data by night, adjusting the timestamp if necessary, and creates a unique `night_group` identifier for each night. Additionally, a cumulative step count (norm_step) is computed for each group to facilitate sequential analysis.

- `transform_series`: this function enhances the transform function by adding a new feature: detecting clipped ENMO values. It flags instances where the enmo (motion metric) is zero, marking potential data quality issues.

- `transform_events`: this function processes event data by adding a night column and pivoting the data. The events are rearranged by `series_id`, `group_id`, and night to simplify time-series analysis.

- `add_feature`: this function generates advanced features for sleep detection, including:
    - Difference Features: Computes the differences in anglez (angular motion) and enmo (motion magnitude).
    - Rolling Median: Calculates rolling medians of anglez_diff and enmo_diff over a 5-minute window.
    - Critical Threshold: Determines static periods by evaluating anglez_diff variability over a day.
    - Static and Sleep Blocks: Flags periods with minimal motion (is_static) and identifies sleep blocks over 30-minute windows.
    - Sleep Episodes: Detects continuous sleep episodes, identifies the longest one, and flags interruptions in sleep.

-  `create_heuristic`: the main function processes raw data files by converting timestamps and applying transformations. It calls the transform_series function to prepare the data and the add_feature function to generate sleep-related features. Finally, it saves the processed data into .parquet files for further analysis.

In [ ]:
MAX_FILE = 2000

In [ ]:
def transform(df, night_offset=20):
    return (
        df.with_columns(
            [
                (pl.col("timestamp").dt.year() - 2000).cast(pl.Int8).alias("year"),
                pl.col("timestamp").dt.month().cast(pl.Int8).alias("month"),
                pl.col("timestamp").dt.day().cast(pl.Int8).alias("day"),
                pl.col("timestamp").dt.hour().cast(pl.Int8).alias("hour"),
                pl.col("timestamp").dt.minute().cast(pl.Int8).alias("minute"),
                pl.col("timestamp").dt.second().cast(pl.Int8).alias("second"),
                pl.col("timestamp").dt.weekday().cast(pl.Int8).alias("weekday"),
            ]
        )
        .with_columns( 
            pl.when(pl.col("hour") < night_offset)
            .then(pl.col("timestamp"))
            .otherwise(pl.col("timestamp") + pl.duration(days=1))
            .dt.date()
            .alias("night_group"),
        )
        .with_columns(
            [
                (
                    pl.col("series_id") + pl.lit("_") + pl.col("night_group").cast(pl.Datetime).dt.strftime("%Y%m%d")
                ).alias("group_id"),
            ]
        )
        .with_columns(
            [
                pl.col("timestamp").cum_count().over("group_id").alias("norm_step"),
            ]
        )
        .drop(["night_group"])
    )


def transform_series(df):
    return transform(df).with_columns(
        [
            (pl.col("enmo") == 0).alias("is_enmo_clipped"),
        ]
    )


def transform_events(df):
    return (
        transform(df)
        .with_columns(
            [
                pl.col("night").cast(pl.UInt32).alias("night"),
            ]
        )
        .pivot(["step", "timestamp", "tz_offset"], ["series_id", "group_id", "night"], "event")
    )


def add_feature(
    df,
    day_group_col="group_id",
    term1=(5 * 60) // 5,
    term2=(30 * 60) // 5,
    term3=(60 * 60) // 5,
    min_threshold=0.005,
    max_threshold=0.04,
    center=True,
):
    return (
        df.with_columns(
            [
                pl.col("anglez").diff(1).abs().alias("anglez_diff"),
                pl.col("enmo").diff(1).abs().alias("enmo_diff"),
            ]
        )
        .with_columns(
            [
                pl.col("anglez_diff")
                .rolling_median(term1, center=center)  # 5 min window
                .alias("anglez_diff_median_5min"),
                pl.col("enmo_diff")
                .rolling_median(term1, center=center)  # 5 min window
                .alias("enmo_diff_median_5min"),
            ]
        )
        .with_columns(
            [
                pl.col("anglez_diff_median_5min")
                .quantile(0.1)
                .clip(min_threshold, max_threshold)
                .over(day_group_col)
                .alias("critical_threshold")
            ]
        )
        .with_columns([(pl.col("anglez_diff_median_5min") < pl.col("critical_threshold") * 15).alias("is_static")])
        .with_columns(
            [
                pl.col("is_static").cast(pl.Int32).rolling_sum(term2, center=center).alias("is_static_sum_30min"),
            ]
        )
        .with_columns([(pl.col("is_static_sum_30min") == ((30 * 60) // 5)).alias("tmp")])
        .with_columns(
            [
                pl.col("tmp").shift(term2 // 2).alias("tmp_left"),
                pl.col("tmp").shift(-(term2 // 2)).alias("tmp_right"),
            ]
        )
        .with_columns(
            [
                (pl.col("tmp_left") | pl.col("tmp_right")).alias("is_sleep_block"),
            ]
        )
        .drop(["tmp", "tmp_left", "tmp_right"])
        .with_columns([pl.col("is_sleep_block").not_().alias("is_gap")])
        .with_columns([pl.col("is_gap").cast(pl.Int32).rolling_sum(term3, center=center).alias("gap_length")])
        .with_columns([(pl.col("gap_length") == term3).alias("tmp")])
        .with_columns(
            [
                pl.col("tmp").shift(term3 // 2).alias("tmp_left"),
                pl.col("tmp").shift(-(term3 // 2)).alias("tmp_right"),
            ]
        )
        .with_columns(
            [
                (pl.col("tmp_left") | pl.col("tmp_right")).alias("is_large_gap"),
            ]
        )
        .drop(["tmp", "tmp_left", "tmp_right"])
        .with_columns([pl.col("is_large_gap").not_().alias("is_sleep_episode")])
        #
        # extract longest sleep episode
        #
        .with_columns(
            [
                # extract false->true transition
                (
                    (
                        pl.col("is_sleep_episode")
                        & pl.col("is_sleep_episode").shift(1, fill_value=pl.lit(False)).not_()
                    )
                    .cum_sum()
                    .over("group_id")
                ).alias("sleep_episode_id")
            ]
        )
        .with_columns(
            [pl.col("is_sleep_episode").sum().over(["group_id", "sleep_episode_id"]).alias("sleep_episode_length")]
        )
        .with_columns([pl.col("sleep_episode_length").max().over(["group_id"]).alias("max_sleep_episode_length")])
        .with_columns(
            [
                (
                    pl.col("is_sleep_episode") & (pl.col("sleep_episode_length") == pl.col("max_sleep_episode_length"))
                ).alias("is_longest_sleep_episode")
            ]
        )
    )


use_columns = [
    "series_id",
    "step",
    "is_longest_sleep_episode",
    "is_sleep_block",
    "is_gap",
    "is_large_gap",
    "is_sleep_episode",
    "is_static",
]

def create_heuristic(paths, train_or_test):
    i = 0
    for path in tqdm(paths):
        i += 1
        if (i == MAX_FILE):
            break
        sdf = pl.read_parquet(path)
    
        # dummy timestamp
        sdf = sdf.with_columns((pl.col("time_of_day") == 0).cast(pl.Int32).cum_sum().alias("day_offset"))
        sdf = sdf.with_columns(
            (
                datetime.datetime(2020, 1, 1)
                + (pl.col("day_offset") * 86400_000_000 + pl.col("time_of_day") / 1000).cast(pl.Duration("us"))
            ).alias("timestamp")
        )
    
        sdf = sdf.with_columns(pl.lit(path.split("/")[-2]).alias("series_id"))
        sdf = sdf.sort("step")
        sdf = transform_series(sdf)
        sdf = add_feature(sdf)
        sdf = sdf[use_columns].fill_null(False)
    
        sidf = path.split("/")[-2]
        save_path = f"/kaggle/working/heuristic_features/{train_or_test}/{sidf}.parquet"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        sdf.write_parquet(save_path)

In [ ]:
if True:
    sys.path.append("/kaggle/input/cmi-2023-src")
    from consts import ANGLEZ_MEAN, ANGLEZ_STD, ENMO_MEAN, ENMO_STD
    from torch_models.dataset import ZzzPatchDataset
    from torch_models.models import ZzzConv1dGRUModel, ZzzTransformerGRUModel, ZzzWaveGRUModel

    from utils.feature_contena import Features
    from utils.lightning_utils import MyLightningDataModule, MyLightningModule
    from utils.set_seed import seed_base_torch
    from utils.torch_template import EnsembleModel

In [ ]:
def detection(paths=f"/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=*/part-0.parquet", train_or_test="train"):
    MODEL_NAME = "patch_transformer_gru"
    
    PACKAGE_DIR = Path("/kaggle/input/cmi-2023-src")
    CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
    BLOCK_SIZE = CFG[MODEL_NAME]["execution"]["block_size"]
    
    CFG["output_dir"] = f"/kaggle/input/cmi-2023-output/{CFG[MODEL_NAME]['execution']['best_exp_id']}"
    
    seed_base_torch(CFG["env"]["seed"])
    
    DEVICE = "cuda"
    
    files = glob(
        paths
    )
    
    features = Features()
    features.add_num_features(["anglez", "enmo"])
    features.add_num_features(["anglez_diff", "enmo_diff"])
    features.add_num_features(["same_count"])
    features.add_num_features(["large_diff_count"])
    features.add_num_features(["same_count_shift_plus", "same_count_shift_minus"])
    features.add_num_features(["is_longest_sleep_episode", "is_sleep_block"])
    
    # transformer + gru
    model = ZzzTransformerGRUModel(
        max_len=BLOCK_SIZE // CFG[MODEL_NAME]["execution"]["patch_size"],
        input_numerical_size=len(features.all_features()) * CFG[MODEL_NAME]["execution"]["patch_size"],
        **CFG[MODEL_NAME]["params"],
    )
    trn_models = [
        MyLightningModule.load_from_checkpoint(
            os.path.join("/kaggle/input/cmi-2023-output/exp_160", f"logs/best_model_fold{fold}.ckpt"),
            model=model,
            map_location=torch.device(DEVICE),
        ).to(DEVICE)
        for fold in range(5 if len(files) > 100 else 1)
    ]
    
    models = trn_models
    model = EnsembleModel(models).to(DEVICE)
    model.eval()
    
    all_oof_dfs = []
    i = 0
    for file in tqdm(files):
        # load file
        i += 1
        if (i == MAX_FILE):
            break
        df = pd.read_parquet(file)
        if len(df) < BLOCK_SIZE:
            continue
        time_of_days = df["time_of_day"].values
    
        # same_count
        DAY_STEPS = 12 * 60 * 24
        n_days = int(len(df) // DAY_STEPS) + 1
        df["same_count"] = 0
        for day in range(-n_days, n_days + 1):
            if day == 0:
                continue
            df["_anglez_diff"] = df["anglez"].diff(DAY_STEPS * day)
            df["_anglez_diff"] = df["_anglez_diff"].fillna(1)
            df["same_count"] += (df["_anglez_diff"] == 0).astype(int)
        df["same_count"] = (df["same_count"].clip(0, 5) - 2.5) / 2.5
    
        SHIFT_STEPS = 12 * 60 * 6  # 6h
        df["same_count_shift_plus"] = df["same_count"].shift(SHIFT_STEPS).fillna(1.0).astype(np.float16)
        df["same_count_shift_minus"] = df["same_count"].shift(-SHIFT_STEPS).fillna(1.0).astype(np.float16)
    
        # features
        df["anglez_diffabs"] = df["anglez"].diff().abs().fillna(0)
        df["large_diff"] = (df["anglez_diffabs"] > 5).astype(int)
        df["large_diff_count"] = df["large_diff"].rolling(10, center=True).mean().fillna(0)
        df["large_diff_count"] = (df["large_diff_count"] - 0.5) * 2
    
        # normalize
        df["anglez"] = (df["anglez"] - ANGLEZ_MEAN) / ANGLEZ_STD
        df["enmo"] = (df["enmo"] - ENMO_MEAN) / ENMO_STD
        df["anglez_diff"] = df["anglez"].diff().fillna(0)
        df["enmo_diff"] = df["enmo"].diff().fillna(0)
    
        # heuristic_features by @bilzard
        sid = file.split("/")[-2]
        df["series_id"] = sid
        path = f"/kaggle/working/heuristic_features/{train_or_test}/{sid}.parquet"
        hdf = pd.read_parquet(path)
        df = pd.concat([df, hdf.drop(columns=["series_id", "step"])], axis=1)
        df[["is_longest_sleep_episode", "is_sleep_block"]] = df[["is_longest_sleep_episode", "is_sleep_block"]] * 2 - 1
    
        # split
        dfs = []
        df = df.sort_values("step").reset_index(drop=True)
        for start in range(0, len(df), BLOCK_SIZE // 8):
            end = start + BLOCK_SIZE
            if end > len(df):
                end = len(df) - len(df) % CFG[MODEL_NAME]["execution"]["patch_size"]
                start = end - BLOCK_SIZE
                assert start >= 0
            assert df.iloc[start]["step"] % CFG[MODEL_NAME]["execution"]["patch_size"] == 0
            dfs.append(df.iloc[start:end])
        gc.collect()
    
        # inference
        train_dataset = ZzzPatchDataset(
            dfs, mode="test", features=features, patch_size=CFG[MODEL_NAME]["execution"]["patch_size"]
        )
        valid_dataset = ZzzPatchDataset(
            dfs, mode="test", features=features, patch_size=CFG[MODEL_NAME]["execution"]["patch_size"]
        )
        data_module = MyLightningDataModule(train_dataset, valid_dataset, batch_size=64)
        preds = []
        with torch.no_grad():
            for X in data_module.val_dataloader():
                pred = torch.sigmoid(model(X.to("cuda"))).detach().cpu().numpy() * 10
                preds.append(pred)
    
        oof_dfs = []
        for pred, df in zip(np.vstack(preds), dfs):
            df = df.iloc[
                CFG[MODEL_NAME]["execution"]["patch_size"] // 2 : len(df) : CFG[MODEL_NAME]["execution"]["patch_size"]
            ].reset_index(drop=True)
            df[["wakeup_oof", "onset_oof"]] = pred
            oof_dfs.append(df[["series_id", "step", "wakeup_oof", "onset_oof"]])
    
        oof_df = pd.concat(oof_dfs)
        oof_df = oof_df.groupby(["series_id", "step"]).mean().reset_index().sort_values(["series_id", "step"])
        oof_df = oof_df[["series_id", "step", "wakeup_oof", "onset_oof"]]
        oof_df["step"] = oof_df["step"].astype(int)
    
        del preds, oof_dfs
        gc.collect()
    
        train = oof_df.reset_index(drop=True)
        train["time_of_day"] = time_of_days[
            CFG[MODEL_NAME]["execution"]["patch_size"] // 2 :: CFG[MODEL_NAME]["execution"]["patch_size"]
        ][: len(train)]
        all_oof_dfs.append(train[["series_id", "step", "wakeup_oof", "onset_oof", "time_of_day"]])
        # del dfs, df
        gc.collect()

    # save
    for df in tqdm(all_oof_dfs):
        save_path = f"/kaggle/working/features/sleep_detection/{train_or_test}/{df['series_id'].iloc[0]}.parquet"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        df.to_parquet(save_path, index=False)

In [ ]:
time_of_day_max = 86400000000000

In [ ]:
def feature_engineering_sleep_detect(paths="/kaggle/working/features/sleep_detection/train/*.parquet", data_paths="/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet", train_or_test="train"):
    features = []
    debug_count = 0
    all_files = sorted(glob(paths))
    i = 0
    for file in tqdm(all_files):
        i += 1
        if (i == MAX_FILE):
            break
        df = pl.read_parquet(file)
        df = df.with_columns(pl.col("step").cast(pl.UInt32)).drop("time_of_day")
        sid = df["series_id"][0]
    
        sensor_df = pl.read_parquet(
            f"{data_paths}/{sid}/part-0.parquet"
        ).with_columns((pl.col("time_of_day") == 0).cum_sum().alias("day"))
    
        feature = {
            "id": sid,
            "length": df.shape[0],
            "day": sensor_df["relative_date_PCIAT"].max() - sensor_df["relative_date_PCIAT"].min(),
        }
    
        # skip if time step is not 5sec
        diffs = sensor_df["time_of_day"].diff().drop_nulls().unique()
        if set(diffs) != set([-86395000000000, 5000000000]):
            features.append(feature)
            continue
    
        sensor_df = (
            sensor_df.join(df, on="step", how="left")
            .sort("step")
            .with_columns(
                pl.col("onset_oof").interpolate(),
                pl.col("wakeup_oof").interpolate(),
            )
        )
    
        # onset = 15:00~3:00, wakeup = 3:00~15:00
        onset_start = time_of_day_max / 24 * 15  # 15:00
        onset_end = time_of_day_max / 24 * 3  # 3:00
        sensor_df = sensor_df.with_columns(
            ((pl.col("time_of_day") > onset_start) | (pl.col("time_of_day") < onset_end)).alias("onset_duration"),
        ).with_columns(
            pl.col("onset_duration").cast(pl.Int32).diff().fill_null(0).abs().cum_sum().alias("onset_wakeup_duration")
        )
    
        # get sleep period
        sleep_info = []
        for _, df in sensor_df.group_by("onset_wakeup_duration", maintain_order=True):
            is_onset = df["onset_duration"][0]
            if is_onset:
                max_idx = df["onset_oof"].arg_max()
                if max_idx is None:
                    continue
                max_score = df["onset_oof"][max_idx]
                step = df["step"][max_idx]
    
                # date
                start_time = df["time_of_day"][0] / time_of_day_max * 24
                if start_time >= 15:
                    day = df["day"][0]
                    week_day = df["weekday"][0]
                else:
                    day = df["day"][0] - 1
                    week_day = df["weekday"][0] - 1
                    if week_day == 0:
                        week_day = 7
            else:
                max_idx = df["wakeup_oof"].arg_max()
                if max_idx is None:
                    continue
                max_score = df["wakeup_oof"][max_idx]
                step = df["step"][max_idx]
    
                # date
                start_time = df["time_of_day"][0] / time_of_day_max * 24
                day = df["day"][0] - 1
                week_day = df["weekday"][0] - 1
    
            info = {
                "day": day,
                "weekday": week_day,
                "type": "onset" if is_onset else "wakeup",
                "step": step,
                "max_score": max_score,
                "time": df["time_of_day"][max_idx] / time_of_day_max * 24,
            }
            sleep_info.append(info)
        sleep_df = pl.DataFrame(sleep_info)
    
        # merge
        sleep_df = (
            sleep_df.filter(pl.col("type") == "onset")
            .drop("type")
            .rename(
                {
                    "max_score": "onset_score",
                    "step": "onset_step",
                    "time": "onset_time",
                }
            )
            .join(
                sleep_df.filter(pl.col("type") == "wakeup")
                .drop(["type", "weekday"])
                .rename(
                    {
                        "max_score": "wakeup_score",
                        "step": "wakeup_step",
                        "time": "wakeup_time",
                    }
                ),
                on="day",
            )
        ).select(
            ["day", "weekday", "onset_time", "wakeup_time", "onset_step", "wakeup_step", "onset_score", "wakeup_score"]
        )
    
        # feature engineering
        sleep_lengths = []  # wakeup - onset
        sleep_enmo_mean = []  
        sleep_enmo_std = []  
        sleep_light_mean = []
        sleep_light_std = [] 
        for i in range(len(sleep_df)):
            # sleep period
            start = sleep_df["onset_step"][i]
            end = sleep_df["wakeup_step"][i]
            if sleep_df["onset_score"][i] < 1 or sleep_df["wakeup_score"][i] < 1:
                sleep_lengths.append(np.nan)
                sleep_enmo_mean.append(np.nan)
                sleep_enmo_std.append(np.nan)
                sleep_light_mean.append(np.nan)
                sleep_light_std.append(np.nan)
                continue
    
            # sleep length
            length = end - start
            sleep_lengths.append(length * 5 / 60 / 60)  # hour
    
            # enmo
            enmo_mean = sensor_df["enmo"][start:end].mean()
            enmo_std = sensor_df["enmo"][start:end].std()
            sleep_enmo_mean.append(enmo_mean)
            sleep_enmo_std.append(enmo_std)
    
            # light
            light_mean = sensor_df["light"][start:end].mean()
            light_std = sensor_df["light"][start:end].std()
            sleep_light_mean.append(light_mean)
            sleep_light_std.append(light_std)
            
        sleep_df = sleep_df.with_columns(
            pl.DataFrame(
                {
                    "sleep_length": sleep_lengths,
                    "sleep_enmo_mean": sleep_enmo_mean,
                    "sleep_enmo_std": sleep_enmo_std,
                    "sleep_light_mean": sleep_light_mean,
                    "sleep_light_std": sleep_light_std,
                }
            )
        )
        
        # leave only high confidence periods
        sleep_df = sleep_df.filter((pl.col("wakeup_score") > 1) & (pl.col("onset_score") > 1))
        if debug_count < 3:
            display(sleep_df.head())
        debug_count += 1
            
    
        # agg
        feature.update(
            {
                "sleep_measurement_count": sleep_df.shape[0],
                "sleep_length_mean": sleep_df["sleep_length"].mean(),
                "sleep_length_std": sleep_df["sleep_length"].std(),
                "sleep_start_mean": sleep_df["onset_time"].mean(),
                "sleep_start_std": sleep_df["onset_time"].std(),
                "sleep_end_mean": sleep_df["wakeup_time"].mean(),
                "sleep_end_std": sleep_df["wakeup_time"].std(),
                "sleep_enmo_mean_mean": sleep_df["sleep_enmo_mean"].mean(),
                "sleep_enmo_mean_std": sleep_df["sleep_enmo_mean"].std(),
                "sleep_enmo_std_mean": sleep_df["sleep_enmo_std"].mean(),
                "sleep_enmo_std_std": sleep_df["sleep_enmo_std"].std(),
                "sleep_light_mean_mean": sleep_df["sleep_light_mean"].mean(),
                "sleep_light_mean_std": sleep_df["sleep_light_mean"].std(),
                "sleep_light_std_mean": sleep_df["sleep_light_std"].mean(),
                "sleep_light_std_std": sleep_df["sleep_light_std"].std(),
            }
        )
        features.append(feature)
    output_dir = f"/kaggle/working/features/{train_or_test}"
    os.makedirs(output_dir, exist_ok=True)
    feature_df = pl.DataFrame(features).with_columns(pl.col("id").str.slice(3, 8))
    feature_df.write_csv(f"/kaggle/working/features/{train_or_test}/sleep_features.csv")
    print(feature_df.head())

In [ ]:
create_heuristic(paths=glob("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=*/part-0.parquet"), train_or_test="test")
detection(paths="/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=*/part-0.parquet", train_or_test="test")
feature_engineering_sleep_detect(paths="/kaggle/working/features/sleep_detection/test/*.parquet", data_paths="/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet", train_or_test="test")

### Utils functions

In [6]:
# Handle non-numeric columns
def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

# Download timeseries data
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

# Function to evaluate the predictions and optimize the thresholds
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

## Function to train the model with processed time series data

In [7]:
def TrainML(model_class, X, y, test_data):
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)
    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    print('OPTIMIZED THRESHOLDS', KappaOPtimizer.x)
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })
    optimized_thresholds = KappaOPtimizer.x
    return submission, oof_tuned, oof_non_rounded, y, optimized_thresholds

# Define features

## Normal features

In [8]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

total_features = list(test.columns)
total_features.remove('id')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

In [9]:
noseason_features = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total',
                'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
                'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
                'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW','BMI_PHR']

64


## Loading timeseries

In [10]:
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00, 12.14it/s]


In [11]:
df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)
autoencoder, scaler = perform_autoencoder(df_train, encoding_dim=60, epochs=100, batch_size=32)

Epoch [50/100], Loss: 1.4613]
Epoch [100/100], Loss: 1.4550]


In [12]:
train_ts_encoded = encode_data(autoencoder, scaler, df_train)
test_ts_encoded = encode_data(autoencoder, scaler, df_test)
test_ts_encoded.reset_index(inplace=True, drop=True)

In [13]:
train_ts_encoded["id"]=train_ts["id"]
test_ts_encoded['id']=test_ts["id"]

## Additiontal timeseries features

In [14]:
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")
time_encoded_cols = train_ts_encoded.columns.tolist()
time_encoded_cols.remove("id")

In [ ]:
train_sleep = pd.read_csv("/kaggle/input/sleep-detection/sleep_features.csv")
test_sleep = pd.read_csv("/kaggle/working/features/test/sleep_features.csv")

In [ ]:
sleep_cols = train_sleep.columns.tolist()
sleep_cols.remove("id")

In [ ]:
rm -rf /kaggle/working/features

In [ ]:
rm -rf /kaggle/working/heuristic_features

# Submission

In [15]:
train_sub1 = pd.merge(train, train_ts_encoded, how="left", on='id')
test_sub1 = pd.merge(test, test_ts_encoded, how="left", on='id')
train_sub1 = train_sub1.dropna(subset='sii')

In [16]:
X_sub1 = train_sub1
y_sub1 = train_sub1['sii']

In [69]:
train_sub2 = pd.merge(train, train_sleep, how="left", on='id')
test_sub2 = pd.merge(test, test_sleep, how="left", on='id')


# pipeline 
train_sub2 = feature_engineering_tabular(train_sub2)
train_sub2 = train_sub2.dropna(subset='sii', ignore_index=True)
test_sub2 = feature_engineering_tabular(test_sub2)

train_sub2 = train_sub2.drop('id', axis=1)
test_sub2  = test_sub2.drop('id', axis=1)   

In [70]:
features_sub2 = noseason_features + sleep_cols

train_sub2 = pd.merge(train, train_ts, how="left", on='id')
test_sub2 = pd.merge(test, test_ts, how="left", on='id')

train_sub2 = train_sub2.dropna(subset='sii')
if np.any(np.isinf(train_sub2)):
    train_sub2 = train_sub2.replace([np.inf, -np.inf], np.nan)

X_sub2 = train_sub2[features_sub2]
y_sub2 = train_sub2['sii']
test_sub2 = test_sub2[features_sub2]


## Tuning

In [ ]:
def objective_sub(trial):
    CatBoost_Params = {
        'learning_rate': trial.suggest_float('catboost_learning_rate', 1e-3, 0.3, log=True),
        'depth': trial.suggest_int('catboost_depth', 4, 10),
        'random_seed': SEED,
        'verbose': 0,
        'l2_leaf_reg': trial.suggest_float('catboost_l2_leaf_reg', 0.01, 10.0, log=True),
        'iterations': trial.suggest_int('catboost_iterations', 100, 400, 10),
        'task_type': 'GPU',  
        'devices': '0'      
    }
    XGB_Params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 500, 1500, 100),
        'max_depth': trial.suggest_int('xgb_max_depth', 1, 10),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('xgb_subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.05, 1.0),
        'gamma': trial.suggest_float('xgb_gamma', 1e-2, 1.0),
        'min_child_weight': trial.suggest_int('xgb_min_child_weight', 1, 100),
        'eval_metric': 'rmse',
        'objective': 'reg:squarederror',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'gpu_id': 0
    }
    LightGBM_Params = {
        'max_depth': trial.suggest_int('lightgbm_max_depth', 3, 12),  # Avoid overly shallow or deep trees
        'min_data_in_leaf': trial.suggest_int('lightgbm_min_data_in_leaf', 5, 50),  # Balance between overfitting and splits
        'num_leaves': trial.suggest_int('lightgbm_num_leaves', 16, 256),  # Limit complexity to avoid splits failing
        'learning_rate': trial.suggest_float('lightgbm_learning_rate', 0.01, 0.1),
        'feature_fraction': trial.suggest_float('lightgbm_feature_fraction', 0.7, 1.0),  # Allow feature subsampling tuning
        'bagging_fraction': trial.suggest_float('lightgbm_bagging_fraction', 0.7, 1.0),  # Improve generalization
        'bagging_freq': trial.suggest_int('lightgbm_bagging_freq', 1, 5),
        'lambda_l1': trial.suggest_float('lightgbm_lambda_l1', 0.0, 10.0),  # Regularization tuning
        'lambda_l2': trial.suggest_float('lightgbm_lambda_l2', 0.0, 10.0),
        'min_gain_to_split': trial.suggest_float('lightgbm_min_gain_to_split', 0.0, 0.1),  # Ensure splits happen
        'device_type': 'gpu',
        'gpu_device_id': 0,
        'verbosity': -1  
    }
    LightGBM_Model = LGBMRegressor(**LightGBM_Params)
    XGB_Model = XGBRegressor(**XGB_Params)
    CatBoost_Model = CatBoostRegressor(**CatBoost_Params)
    voting_model = VotingRegressor(estimators=[
        ('lightgbm', LightGBM_Model),
        ('xgboost', XGB_Model),
        ('catboost', CatBoost_Model),
    ], weights=[4.0, 4.0, 4.0, 5.0])
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    submission1, val_score, _, _, _, _ = TrainML(voting_model, X_sub1, y_sub1, test_sub1)
    return val_score

In [17]:
SVR_Best_Params = {
    'C': 0.1,
    'epsilon': 0.1,
    'kernel': 'rbf',
    'gamma': 'scale',
}

CatBoost_Best_Params = {
    'learning_rate': 0.0021172579310639343,
    'depth': 6,
    'iterations': 130,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 0.32557701990001503,
    'task_type': 'GPU',  
    'devices': '0'
}

XGB_Best_Params = {
    'n_estimators': 700,
    'max_depth': 4,
    'learning_rate': 0.03325152156380898,
    'subsample': 0.25295047248406266,
    'colsample_bytree': 0.9760859719849787,
    'gamma': 0.20085951790463402,
    'min_child_weight': 11,
    'eval_metric': 'rmse',
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'gpu_id': 0
}

LightGBM_Best_Params = {
    'max_depth': 3,
    'min_data_in_leaf': 40,
    'num_leaves': 190,
    'learning_rate': 0.05107368421432176,
    'feature_fraction': 0.9918350138636185,
    'bagging_fraction': 0.9331400899763774,
    'bagging_freq': 1,
    'lambda_l1': 9.49641646280519,
    'lambda_l2': 2.446305429623661,
    'min_gain_to_split': 0.05262124930522051,
    'device_type': 'gpu',
    'gpu_device_id': 0,
    'verbosity': -1
}



catboost_model = CatBoostRegressor(**CatBoost_Best_Params)
xgb_model = XGBRegressor(**XGB_Best_Params)
lightgbm_model = LGBMRegressor(**LightGBM_Best_Params)
svr_model = SVR(**SVR_Best_Params)

final_voting_model = VotingRegressor(estimators=[
    ('lightgbm', lightgbm_model),
    ('xgboost', xgb_model),
    ('catboost', catboost_model),
], weights=[4.0, 4.0, 4.0])

X = train.drop(['sii'], axis=1)
y = train['sii']

In [18]:
submission1, val_score_sub1, _, _, _, _ = TrainML(xgb_model, X_sub1, y_sub1, test_sub1)

print("Val score sub1 with best parameters:", val_score_sub1)

Training Folds: 100%|██████████| 5/5 [00:12<00:00,  2.53s/it]

Mean Train QWK --> 0.5441
Mean Validation QWK ---> 0.3645


OPTIMIZED THRESHOLDS [0.5785469  0.88500199 2.83677574]
----> || Optimized QWK SCORE ::  0.462
Val score sub1 with best parameters: 0.46241743563848414


# Final

In [ ]:
final_submission = submission1
final_submission.to_csv('submission.csv', index=False)

print("Submission saved to 'submission.csv'")

In [ ]:
final_submission